In [3]:
from langchain_groq import ChatGroq
import os

In [10]:
import chromadb

In [9]:
GROG_API_KEY = os.environ.get("MY_GROG_API_KEY")

In [6]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=None,
    max_retries=2
)

In [15]:
client = chromadb.Client()
#collection = client.create_collection(name="cold_mail")
collection = client.create_collection(name="cold_email")

In [ ]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Dhaka,Bangladesh"
    ],
    ids=["id1","id2"]
)

In [ ]:
all_docs = collection.get(ids=["id1"])
all_docs

In [22]:
results = collection.query(
    query_texts=['Query is about Bagels'],
    n_results=1
)
results

{'ids': [['id1']],
 'distances': [[1.7323635816574097]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [['This document is about New York']],
 'uris': None,
 'data': None}

In [29]:
collection.delete(ids=["id2"])
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

Capturing Website Data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-43863")
page_data = loader.load().pop().page_content
#print(page_data)

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """ 
    ### SRAPPED TEST FROM WEBSITE
    {page_date}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is extract the job postings and return them in JSON format containing following keys: 'role','experience','skills' and 'description'
    Only return valid JSON.
    ### VALID JSON (NO PREABLE)
    """
)